In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from keras.utils import to_categorical

import numpy as np
from tensorflow.keras.optimizers import RMSprop

Using TensorFlow backend.


In [36]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.reshape(x_train.shape[0], 784), x_test.reshape(x_test.shape[0], 784)
#normalise:
x_train, x_test = x_train/255, x_test/255

In [8]:
encoded_dim = 32 #compression factor = 784/32 = 24.5
regulariser = keras.regularizers.l2(0.01)

input_img = keras.Input(shape=(784,))

In [20]:
#Encoded layer:
encoded = layers.Dense(encoded_dim, activation="relu", kernel_regularizer = regulariser)(input_img)

#decoded layer:
decoded = layers.Dense(784, activation = "linear", kernel_regularizer = regulariser)(encoded)

#model:
autoencoder = keras.Model(input_img, decoded)


encoder = keras.Model(input_img, encoded)
encoder_input = keras.Input(shape=(encoded_dim,))

decoder_layer = autoencoder.layers[-1]
decoder = keras.Model(encoder_input, decoder_layer(encoder_input))

autoencoder.compile(optimizer='sgd', loss='binary_crossentropy')

In [21]:
epochs = 50 
batch_size = 256

autoencoder.fit(x_train, x_train,epochs=epochs, batch_size=batch_size, shuffle=True, validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
60000/60000 [==============================] - 3s 49us/sample - loss: 1.9239 - val_loss: 1.6652
Epoch 2/50
60000/60000 [==============================] - 2s 38us/sample - loss: 1.5431 - val_loss: 1.4588
Epoch 3/50
60000/60000 [==============================] - 2s 41us/sample - loss: 1.4011 - val_loss: 1.3462
Epoch 4/50
60000/60000 [==============================] - 2s 39us/sample - loss: 1.2943 - val_loss: 1.2420
Epoch 5/50
60000/60000 [==============================] - 2s 38us/sample - loss: 1.1990 - val_loss: 1.1541
Epoch 6/50
60000/60000 [==============================] - 2s 38us/sample - loss: 1.1126 - val_loss: 1.0695
Epoch 7/50
60000/60000 [==============================] - 2s 37us/sample - loss: 1.0288 - val_loss: 0.9826
Epoch 8/50
60000/60000 [==============================] - 2s 37us/sample - loss: 0.9505 - val_loss: 0.9198
Epoch 9/50
60000/60000 [======================

In [37]:
encoded_train = encoder.predict(x_train)
encoded_test = encoder.predict(x_test)

y_train, y_test = to_categorical(y_train), to_categorical(y_test)
num_of_classes = len(y_train[0])


In [62]:
keep_prob = 0.8
#Using keras sequential model:
model = keras.Sequential()
model.add(layers.Dense(num_of_classes, activation='relu', input_shape=(32,)))

model.add(layers.Dense(128, activation = 'relu', input_shape=(32,)))
model.add(layers.Dropout(0.2))


model.add(layers.Dense(32, activation = 'relu', ))
model.add(layers.Dropout(0.2))


model.add(layers.Dense(num_of_classes, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])


epochs = 20
model.fit(encoded_train, y_train, epochs=epochs, verbose=1, validation_data=(encoded_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 4s 59us/sample - loss: 1.3393 - acc: 0.4945 - val_loss: 0.9468 - val_acc: 0.6487
Epoch 2/20
60000/60000 [==============================] - 3s 52us/sample - loss: 0.9532 - acc: 0.6509 - val_loss: 0.7980 - val_acc: 0.7088
Epoch 3/20
60000/60000 [==============================] - 3s 54us/sample - loss: 0.8597 - acc: 0.6899 - val_loss: 0.7722 - val_acc: 0.7174
Epoch 4/20
60000/60000 [==============================] - 3s 57us/sample - loss: 0.8056 - acc: 0.7117 - val_loss: 0.7080 - val_acc: 0.7507
Epoch 5/20
60000/60000 [==============================] - 3s 54us/sample - loss: 0.7746 - acc: 0.7264 - val_loss: 0.6866 - val_acc: 0.7540
Epoch 6/20
60000/60000 [==============================] - 3s 54us/sample - loss: 0.7500 - acc: 0.7356 - val_loss: 0.6543 - val_acc: 0.7698
Epoch 7/20
60000/60000 [==============================] - 3s 54us/sample - loss: 0.7316 - acc: 0.7433 - val_loss: 0.

In [69]:
score = model.evaluate(encoded_test, y_test, verbose=0)
print(" Test loss =", score[0],"\n", "Test accuracy =", score[1])

 Test loss = 0.5711329305171966 
 Test accuracy = 0.8015
